In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

In [ ]:
im3 = Image.open(threes[1])
# Image.open(threes[2])
# Image.open(threes[3])
im3

In [ ]:
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[4:26,4:22])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

In [ ]:
three_tensors = [tensor(Image.open(o)) for o in threes]
seven_tensors = [tensor(Image.open(o)) for o in sevens]
# len(three_tensors), len(seven_tensors)
# three_tensors.shape, seven_tensors.shape 

In [ ]:
print(len(three_tensors))
show_image(three_tensors[1])

In [ ]:
# mean calc : required float conversion : general scaling 0 : 1 

stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
print(stacked_threes.shape, stacked_sevens.shape)
print(stacked_threes.ndim )

In [ ]:
mean3 = stacked_threes.mean(0)
mean7 = stacked_sevens.mean(0)
show_image(mean3);
show_image(mean7)

In [ ]:
# MAE : mean absoule error 
# RMSE : root mean of squared error 
dist3_mae = (stacked_threes[0] - mean3).abs().mean()
print('dist3 MAE: ',dist3_mae)
dist3_rmse = ((stacked_threes[0] - mean3)**2).mean().sqrt() 
print('dist3 RMSE: ',dist3_rmse)

In [ ]:
# MAE loss using builtin torch method
F.l1_loss(stacked_threes[0], mean7), F.l1_loss(stacked_threes[0], mean3)

In [ ]:
# RMSE loss using built-in torch method : torch.nn.functional
F.mse_loss(stacked_threes[0], mean7), F.mse_loss(stacked_threes[0], mean3)

In [ ]:
data = [[1,2,3],[4,5,6]]
arr = array (data)
tns = tensor(data)
# arr, tns
# tns[0]
# tns[0,1:]
# tns.type()

tns + 1 , tns * 1.25, tns / 1.1

In [ ]:
# validation tensors from dataset : model evaulation 
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])    # creating a matrix of image tensors
valid_3_tens = valid_3_tens.float()/255    # converting pixel vals as flaot -> scaling to (0->1)
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape

In [ ]:
# fucntion to output tensor : MAE of all images 
def mnist_dist(a,b): return (a-b).abs().mean((-1,-2))
valid_3_dist = mnist_dist(valid_3_tens, mean3)
valid_3_dist, valid_3_dist.shape, (valid_3_tens - mean3).abs().shape 
# options : cosine similarity, RMSE, euclidian distance 

In [ ]:
# function to reutrn pixel similarity to 3 (using MAE) 
def is_3(x) : return mnist_dist(x, mean3) < mnist_dist(x, mean7)

print(is_3(valid_3_tens))

accuracy_3 = is_3(valid_3_tens).float().mean() 
accuracy_7 = (1- is_3(valid_7_tens).float()).mean() 

print('accuracy 3 :', accuracy_3.item())
print('accuracy 7 :', accuracy_7.item())
print('accuracy overall :', torch.stack([accuracy_3, accuracy_7]).mean())

###   machine learning classifier 
1. initialize random wights
2. make predictions -> calculate model accuracy
3. calculate gradient -> calculates change in weights affect on the loss
4. change weights based on gradient calculations
5. re-iterate to step-2

* weight initialize : random weights
* loss : testing the effectiveness of weights
* step : try current weight: increase/decrease weights : calculate direction (inc or dec)
          -> calculating gradient : applying updated weights (inc/dec) : repeat steps
* stop : re-iterate step till : epochs reached, accuracy gets worse, time runs out 
  

In [ ]:
# pytorch gradient calculation 

def f(x) : return x**2  # defining the func  
x = tensor(3.).requires_grad_() # calculate gradient at val -> x
y = f(x)
y.backward() 

x, x.grad, y  # x.grad -> gradient of f(x) : calculated at val (x)

In [ ]:
# using tensor (rank1 : 1 dim-array) this time 

def g(x) : return (x**2).sum()
x = tensor([3.,4.,5.]).requires_grad_()
y = g(x)
y.backward()

x, x.grad, y

In [ ]:
time = torch.arange(0,20).float(); time
speed = torch.randn(20)*3 + 0.75*(time-9.5)**2 + 1

In [ ]:
# SGD using dl/ml 

def f(t, params):
    a,b,c = params
    return a*(t**2) + (b*t) + c

def mse(preds, target) : return ((preds-target)**2).mean()    # loss function 

params = torch.randn(3).requires_grad_()    # tagged to track gradient (weights)
preds = f(time, params)                     # predictions 

def show_preds(preds, ax=None):
    if ax is None: ax=plt.subplots()[1]
    ax.scatter(time, speed)
    ax.scatter(time, to_np(preds), color='red')
    ax.set_ylim(-300,100)

show_preds(preds)
mse(preds, speed)

In [ ]:
# calc loss 
loss = mse(preds, speed)
loss 

# calc gradients 
loss.backward() 
params.grad

In [ ]:
# stepping weights 
lr = 1e-5 
params.data -= params.grad.data * lr 

# params.grad = None
# params.data, params.grad

In [ ]:
# check loss 

preds = f(time,params)
show_preds(preds)
mse(preds, speed), params.data 

* Ist iteration loss -> tensor(53552.4883, grad_fn=<MeanBackward0>)
* 2nd iteration loss -> tensor(10674.2959, grad_fn=<MeanBackward0>)

In [ ]:
def apply_step(params, prnt):
    preds = f(time, params) # predictions 
    loss = mse(preds, speed) # calc loss 
    loss.backward() # calc gradient 
    # stepping weights 
    lr = 1e-5 
    params.data -= params.grad.data * lr 
    params.grad = None 

    if (prnt) : print(loss.item()) 
    return preds 

In [ ]:
# run 10 epochs 
for i in range (10) : apply_step(params, true)

In [ ]:
# plot reducing loss 

_,axs = plt.subplots(1,4,figsize=(12,3))
for ax in axs: show_preds(apply_step(params, False), ax)
plt.tight_layout()

In [ ]:
# MNIST loss function 
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1) # adding labels for each img 
train_x.shape,train_y.shape

In [ ]:
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape,y

In [ ]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(0)
valid_dset = list(zip(valid_x,valid_y))

# initialize random weight for each pixel 
def init_params(size, std=1.0): 
    return (torch.randn(size)*std).requires_grad_()

weights = init_params((28*28,1))
bias = init_params(1)

In [ ]:
# y=w*x+b  : w -> weights , b -> bias , y -> parameter

# backpropagation : error function -> MSE 
# updating weights : hidden layers unlike perceptrons dont have target outputs, as they are intermediate steps to conpute final output  
# backwards propagation -> error terms and derivatives , forward computation -> output 
